The purpose of this notebook is to introduce OOP, and magic functions in Python. B

#### Contents
* Object references are passed by value.
* Introduction to OOP
* Some usecases of OOP
* Python magic functions e.g., `__enter__`, `__exit__`
* Operator overloadding e.g., + with objects.
* Measuring runtime. A class to measure runtime.

## Passing objects to functions

Consider the following two cases. Explain what is going on.

### case 1

In [7]:
def add_more(x):
    x.append(10)

#----------------    
L = [1, 3]
add_more(L)
print L

[1, 3, 10]


### case 2

In [8]:
def inc(x):
    x = x + 1

#-----------------
y = 2
inc(y)
print y

2


In both cases, we modify two variables in place in the function. But, the effect of the modification persists to the caller only in case 1. Why? This has to do with the way **objects** are passed, the way the **reference** holding the object is used. The following code does more or less the same thing as in case 1.

In [9]:
a = [1, 2]
b = a 
b[0] = 10
print a

[10, 2]


There are two things we will need to distinguish: references, and objects. When we write 
        
        a = [1, 2]
        
`a` is a reference. `[1, 2]` is an object. Think of `a` as a hand, and `[1, 2]` as a luggage.

* By `a = [1,2]`, we "ask the hand `a` to hold the luggage `[1, 2]`."
* By `b = a`, we "ask `b` to hold whatever `a` is holding." In this case, `a` is holding the luggage `[1, 2]`. So, `b` then holds the **exact** same luggage `[1, 2]`.
* Basically, the same luggage whose content is `[1, 2]` is held by both hands `a` and `b`.
* By `b[0] = 10`, we change the content of the luggage.
* But, `a` is holding the same luggage. So, accessing the luggage from the hand `a` will also see the change.

In the same way, in case 1, `x` in the function is just another hand holding the luggage `[1, 3]`. That is why.

### What about case 2?

* In the function, `x` holds the luggage `2` which is also held by `y` outside the function.
* Then in the function, `x = x + 1` which is the same as `x = 3`. Here, we "ask the hand `x` to hold a new luggage `3`."
* But this has nothing to do with the hand `y` which is still holding `2`.

### I want to pass [1, 3]. But I don't want my luggage to be modified.

Then, you will need to make a new luggage with the exact same content. 

In [10]:
L = [1, 3]
# A new luggage. ..[:] means "copy".
L2 = L[:]

print L
print L2

[1, 3]
[1, 3]


Basically, up to this point, `L`, `L2` are hands each holding its own luggage. But the luggages happen to look the same.

In [11]:
add_more(L)
print L
print L2

[1, 3, 10]
[1, 3]


## What is OOP

A programming paradigm where code is organized around objects, rather than "actions" (procedure, functions).

* In Matlab, a program is broken down into smaller functions. Data are organized into `struct`, `array` or `cell`s. There is no semantic associated with a piece of data structure. For example, 

        s = struct()
        s.id = 1;
        s.connections = [2,4,8];
        s.threshold = -55;
        s.resting_v = -70
        
    There is nothing saying that `s` represents a neuron. There is no information describing which functions can be used with this piece of data structure.

* In a language with OOP, code is organized around objects. Think of an object as a `struct` with a set of functions that can be used on it. Object = struct + functions. These functions are called *methods*.

* An object always has a type. For example, the above `s` is a `Neuron`. So the **class** of `s` is `Neuron`. This `s` is just a particular realization (or object) from the class `Neuron`. We can create other objects belonging to the same class, each having its own `attributes` (e.g., `threshold`, `resting_v`). As an analogy, we are humans. Each of use is a realization from the class `Human`, carrying different attributes e.g., heights, skin colour.

* All objects in the same class can perform the same kinds of actions, but may yield different results. For instance, all birds can fly but may fly in a different way. All neurons can fire, but perhaps in a different firing patterns, depending on its attributes. *This is to say that a class predefines methods that objects (of that class) can use*.

## Geometric shapes in 2D

Say we want to make a program to process geometric shapes. Let's create a `Circle` class

In [12]:
import math

# This '(object)' will be explained later.
class Circle(object):
    
    # self is like a struct in Matlab. It holds all the attributes.
    # This __init__ is called a  "constructor" because it is used to construct an object.
    # Here we are saying that to construct a circle, we need its center (x,y) and a radius.
    def __init__(self, x, y, radius=1):
        # store the attributes
        self.x = x
        self.y = y
        self.radius = radius
    
    # area is a method that works on only Circle objects.
    def area(self):
        """Compute the area of this circle."""
        a = math.pi*self.radius**2
        return a
        
    

In [13]:
# Let's make a Circle object 

# Circle(..) here will call __init__
c0 = Circle(0, 0, 2)
# radius = 1 by default
c1 = Circle(1, 1)

print c0.area()
print c1.area()

# Can access an attribute as well
print c1.y

12.5663706144
3.14159265359
1


In [14]:
# c0 is a Circle
isinstance(c0, Circle)

True

In [15]:
isinstance('some string', Circle)

False

## Inheritance

Classes can be organized in hierarchy. For example, `Circle` is actually a type of `Shape`. So, we make `Shape` a **superclass** of `Circle`. The class `Circle` is then called a `subclass` of `Shape`.

The main advantage of doing this is to share and enforce implementations of certains methods. As will be seen in the class `Square`.

In [16]:
# Let's define a Shape
class Shape(object):
    # A Shape needs a coordinate
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def area(self):
        # A class where instantiation is not intended is called an abstract class.
        """Subclasses will implement this."""
        raise NotImplementedError('The area of an abstract shape is not known.')

In [17]:
s0 = Shape(1, 2)
# Error
#s0.area()

In [18]:
# Let's make Circle a subclass of Shape.
# Redefine Circle class
class Circle(Shape):
    def __init__(self, x, y, radius=1):
        super(Circle, self).__init__(x, y)
        self.radius = radius
    
    # **override** the area() method
    def area(self):
        """Compute the area of this circle."""
        a = math.pi*self.radius**2
        return a
        

In [19]:
# Same outputs as before
c0 = Circle(0, 0, 2)
c1 = Circle(1, 1)

# No errors when calling area(). 
print c0.area()
print c1.area()

12.5663706144
3.14159265359


In [20]:
# A rectangle is a Shape.
class Rect(Shape):
    
    def __init__(self, x, y, w, h):
        """(x,y) at the upper left corner.
        w: width
        h: height
        """
        super(Rect, self).__init__(x, y)
        self.w = w
        self.h = h
    
    def area(self):
        return self.w * self.h
        

In [21]:
r0 = Rect(0, 0, 2, 3.0)
print r0.area()

6.0


In [22]:
# A Square is a special case of a Rect
class Square(Rect):
    def __init__(self, x, y, length):
        # The constructor of Rect takes 4 inputs.
        # But width = height for a Square
        super(Square, self).__init__(x, y, length, length)
    
    # Don't need to define area(self) because 
    # Square **inherits** area() method from Rect

In [23]:
sq0 = Square(0, 1, 2)
print sq0.area()

4


In [24]:
print isinstance(sq0, Square)
# sq0 is a Rect as well
print isinstance(sq0, Rect)
print isinstance(sq0, Shape)
print isinstance(sq0, Circle)

True
True
True
False


Even in OOP, there are times when it is more convenient to define functions (as in Matlab) independent of any class. For instance, a function that takes in a `Shape` object and saves it. In this case, we can do a simply type checking as follows

In [25]:
def save_shape(s):
    if not isinstance(s, Shape):
        raise ValueError('s has to be a Shape!')
    else:
        # s is a Shape
        # save it
        print 'Saved: %s'%str(s)


In [26]:
save_shape(c0)
# error
#save_shape('I am a string not a Shape')

Saved: <__main__.Circle object at 0x7f0ca82b0dd0>


## Magic functions, magic methods

http://www.rafekettler.com/magicmethods.html 

Special methods in Python that carry special meaning. The names of magic methods are surrounded by `__` (double underscores). We have already seen `__init__()` for an object constructor. 

### `__str__`

`__str__()` is used to convert an object to a string. Useful for quicking printing a summary of a complex object.

In [27]:
class Square(Rect):
    def __init__(self, x, y, length):
        # The constructor of Rect takes 4 inputs.
        # But width = height for a Square
        super(Square, self).__init__(x, y, length, length)

    def __str__(self):
        """Return a string."""
        return 'Square(x=%.2f, y=%.2f, len=%.2f)'%(self.x, self.y, self.w)

In [28]:
sq1 = Square(1, 2, 3)
print sq1
print str(sq1)

Square(x=1.00, y=2.00, len=3.00)
Square(x=1.00, y=2.00, len=3.00)


### `__add__`

Defining `__add__` will define the behaviour of + on object. There are magic methods for -, * as well. They are `__sub__` and `__mul__`. See http://www.rafekettler.com/magicmethods.html for more

In [29]:
class Vector2D(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        # other has to be a Vector2D object
        nx = self.x + other.x
        ny = self.y + other.y
        v = Vector2D(nx, ny)
        return v
    
    def __str__(self):
        return '(%.2f, %.2f)'%(self.x, self.y)

In [30]:
v0 = Vector2D(1, 2)
v1 = Vector2D(1, -2)
# doing v0+v1 actually implicitly returns a new object
print v0 + v1

(2.00, 0.00)


### Note 

* Operator overloading can make the code difficult to read. Use with care. In many cases, simplying doing `a.add(b)` is sufficient (rather than `a + b` with an overloaded `+`).

* numpy arrays are actually objects. So, `a` in `a = np.array([1,2,3])` is an object. Magic methods for indexing with `[..]` are overridden so that we can do `a[0]`.

## Measuring run time

In [31]:
import numpy as np

def waste_time(n):
    """Do an eigen decomposition an nxn matrix"""
    np.random.seed(1)
    
    R = np.random.randn(n, 100)
    A = R.dot(R.T)
    np.linalg.eig(A)
    

In [32]:
import time
# One way to check the run time is
# time.time() returns a time in seconds since the epoch (a fixed reference point in the past) 
# as a floating point number.
start = time.time()
n = 500
waste_time(n)
end = time.time()
diff = end - start
print 'waste_time(%d) took %.3f secs'%(n, diff)

waste_time(500) took 1.312 secs


Note that time spent will also depend on how busy the CPU is at the moment the code is executed. So in practice, the run time is not really deterministic given the input.

## Context manager

http://docs.quantifiedcode.com/python-anti-patterns/correctness/exit_must_accept_three_arguments.html

The following code writes one line to a file.

In [33]:
fname = 'test_file.txt'
f = open(fname, 'w')
f.write('hello')
f.close()

The following code does the exact same thing.

In [34]:
with open(fname, 'w') as f:
    f.write('hello')

What is going on when we write `with ... as ...`? 

When we write `with (c) as (r)`, `(c)` is the so called a **context manager**, and `(r)` is the name of the context manager we want to refer to. Typically, a context manager is some resource that we want to access, and make sure that it is properly closed when we finish. To be able to use `with ... as ...`, `(c)` has to implement `__enter__()` and `__exit__()` magic functions.

* When `with ... as ...` is executed, `__enter__()` is called.
* When the `with ... as ...` block exists, `__exit__()` is called.

In the case, of opening a file its `__exit__()` is implemented to do `f.close()` so we do not need to remember to close.

## A timer class

We can make use of a context manager idea to build a class to time a piece of code:

* `__enter__()` starts the timer.
* `__exit__()` stops the timer.


From https://www.huyng.com/posts/python-performance-analysis


In [35]:
class ContextTimer(object):
    """
    A class used to time an executation of a code snippet. 
    Use it with with .... as ...
    For example, 

        with ContextTimer() as t:
            # do something 
        time_spent = t.secs

    From https://www.huyng.com/posts/python-performance-analysis
    """

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.secs = self.end - self.start 


In [36]:
with ContextTimer() as t:
    waste_time(n)

print 'waste_time(%d) took %.3f secs'%(n, t.secs)

waste_time(500) took 1.050 secs
